In [3]:
import chess.pgn
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to parse PGN file and extract game data
def parse_pgn_file(file_path):
    games = []
    with open(file_path) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games






# Load PGN data
pgn_file = "/Users/ruvinjagoda/Desktop/Aka/AIP/Carlsen.pgn"
games = parse_pgn_file(pgn_file)







In [31]:
# Function to convert board state to one-hot encoding
def board_to_one_hot(board):
    # Initialize an empty array to hold the one-hot encoding
    one_hot = np.zeros((8, 8, 12), dtype=np.int8)
    
    # Define piece mapping
    piece_map = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5
    }
    
    # Iterate over each square on the board
    for square in chess.scan_reversed(chess.BB_ALL):
        # Get the piece on the square
        piece = board.piece_at(square)
        if piece is not None:
            # Determine the index for the piece type and color
            piece_index = piece_map[piece.piece_type] + (6 if piece.color else 0)
            # Set the corresponding element in the one-hot encoding to 1
            one_hot[chess.square_rank(square), chess.square_file(square), piece_index] = 1
    
    return one_hot

# Extract features and labels from all games
X, y = [], []

for game in games:
    
    board = game.board()
    magus_is_white = "magnus" in game.headers["White"].lower()
    for move in game.mainline_moves():
        if magus_is_white:
            
            if board.turn:  # If it's Magnus's turn
                X.append(board_to_one_hot(board))  # Append current board state
                board.push(move)  
                y.append(board_to_one_hot(board))  # Append board state after Magnus's move
            else:  
                board.push(move)
        else:
            
            if not board.turn:  # If it's Magnus's turn
                X.append(board_to_one_hot(board))  # Append current board state
                board.push(move)  
                y.append(board_to_one_hot(board))  # Append board state after Magnus's move
            else:   
                board.push(move)

print(len(X))

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]


# Split data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

260245


In [1]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 12), padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(8 * 8 * 12, activation='softmax'),
    layers.Reshape((8, 8, 12))
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

NameError: name 'models' is not defined

In [37]:
model.save("path_to_model_weights.h5")


In [2]:
import chess
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import pygame

# Function to convert a board state to one-hot encoding
def board_to_one_hot(board):
    one_hot = np.zeros((8, 8, 12), dtype=np.int8)
    piece_map = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5
    }
    for square in chess.scan_reversed(chess.BB_ALL):
        piece = board.piece_at(square)
        if piece is not None:
            piece_index = piece_map[piece.piece_type] + (6 if piece.color else 0)
            one_hot[chess.square_rank(square), chess.square_file(square), piece_index] = 1
    return one_hot

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 12), padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(8 * 8 * 12, activation='softmax'),
    layers.Reshape((8, 8, 12))
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load the pre-trained model weights
model.load_weights("path_to_model_weights.h5")


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
